In [8]:
import qiskit
from qiskit import assemble, QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile
from random import *
from qiskit import Aer
sim = Aer.get_backend("statevector_simulator")
import numpy as np
from qiskit.circuit.library.standard_gates import HGate
from qiskit.circuit.library import RZGate, RYGate


In [22]:
#circuit for periodic 1D chain 
def pbc_qc(i): 
    global spin, angle
    aux1 = spin
    aux2 = spin+1
    qc = QuantumCircuit(spin+2, spin)
    # flip first
    qc.x(i)
    qc.barrier()

    if i == 0: 
        neigh1 = i+1
        neigh2 = aux1-1
        
    elif i == aux1-1:
        neigh1 = i-1
        neigh2 = 0
    else:
        neigh1 = i-1
        neigh2 = i+1

    # deal with the sign with neighbor
    qc.cx(i, aux1), qc.cx(neigh1, aux1)
    qc.cx(i, aux2), qc.cx(neigh2, aux2)
    qc.barrier()

    # both same sign => rotate with acceptance probability

    ### method 1 switch to imaginary part (phase xy-plane), rotate with prob, and then measure at the x-basis 
    # rotate_H = HGate().control(2) 
    # qc.append(rotate_H, [aux2,aux1,i])
    # ccrz = RZGate(angle).control(2,label=None)
    # qc.append(ccrz,[aux2,aux1,i])
    # qc.append(rotate_H, [aux2,aux1,i])

    ### method2 rotated along y-axis and measure at z-basis, these two methods should be equivalent 
    ccry = RYGate(angle).control(2,label=None )
    qc.append(ccry,[aux2,aux1,i])
    
    qc.barrier()
    # measure
    for i in range(spin): 
        qc.measure(i,i)

    return qc

def get_all_qc_pbc(spin):  
    # get all possible circuit
    qc_info=[]
    for i in range(spin):
        qc_info.append(pbc_qc(i))
    return qc_info

def measure_state(qc, sim): 
    qc = transpile(qc,sim)
    final_counts = sim.run(qc, shot=1).result().get_counts()
    return final_counts.keys()

In [23]:
def MC_sweep(start, runs, qc_info, sim):
    global spin
    qc = start
    for run in range(runs):
        # randomly pick site
        pick = randint(1, spin)
        circuit = qc_info[pick-1]
        qc.compose(circuit, inplace=True)
        state = measure_state(qc,sim)
        # run the next MC with previous state
        qc = QuantumCircuit(spin+2, spin)
        for i in state: 
            print(state)
            for j, k in enumerate(i): 
                if k == '1': 
                    qc.x(spin-1-j)
    return state

def map_state(final_state, spin):
    state = np.zeros(spin)
    for i in final_state:
        for j,k in enumerate(i): 
            if k == '1': 
                state[j] = k 
    return state
                

In [24]:
spin = 10
Temp = 0.1
# Temp = 3
prob = np.exp(-4/Temp)
angle = 2*np.arccos(np.sqrt(prob))
qc_info = get_all_qc_pbc(spin)
####initial condition - could be random 
start = QuantumCircuit(spin+2, spin)
start.x(9), start.x(8)
for i in range(2): 
    start.x(i+1)
#####

# run 512 sweeps to equilibrium
final_state = MC_sweep(start,512, qc_info, sim)
state = map_state(final_state, spin)
# print(state)

dict_keys(['1100000111'])
dict_keys(['1000000111'])
dict_keys(['1000000011'])
dict_keys(['1000000011'])
dict_keys(['1000000011'])
dict_keys(['1000000001'])
dict_keys(['0000000001'])
dict_keys(['0000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['0000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000001'])
dict_keys(['1000000000'])
dict_keys(['1000000000'])
dict_keys(['1000000000'])
dict_keys(['1000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['0000000000'])
dict_keys(['